___
___
# **Zonas de oscuridad total durante el eclipse solar 08/04/2024**
**Google Earth Engine**


En este Notebook, trazaremos (burdamente) la trayectoria de la sombra que dejó el eclipse solar sobre el territorio mexicano el día 8 de abril de 2024. También

\


___
___
## **Creando el ambiente de trabajo**
___
___

### **Autenticación**

El primer paso es autorizar este Notebook para poder acceder a la base de datos de Google Earth Engine (GEE).

In [ ]:
import ee

ee.Authenticate()
ee.Initialize(project='san-pinon') # Usar tu project ID

### **Paqueterías necesarias**

La siguiente librería es necesaria para generar mapas.

In [ ]:
import geemap         # Improved mapping library
import pandas as pd

___
___

## **Visualización de imágenes**

___
___

### **Data import**

Lo que sigue es seleccionar la colección de datos satelitales de nuestro interés desde la base de datos de GEE, que en este caso serán los del GOES-16 (región CONUS o FullDisk). Estas imágenes tienen resolución de 2Km p/ píxel.

Para saber el código de la colección de imágenes que quiero exportar necesito buscar en la base de datos de Earth Engine (https://developers.google.com/earth-engine/datasets/catalog)

In [ ]:
# Importa la colección de imagenes satelitales
img_collection = ee.ImageCollection("NOAA/GOES/16/MCMIPC")


Para filtrar esta colección, sólo necesitamos el intervalo de tiempo que nos interesa. No es necesario filtrar por región ya que, al ser un satélite geoestacoinario, siempre captura la misma zona (CONUS/FullDisk en caso del GOES-16).
\
\
Sabemos que el eclipse llegó a las costas de Mazatlán a las 11:10HRS (hora local) y a las 11:30HRS habría salido del país. Hacemos un filtrado inicial utilizando esta información.

In [ ]:
# Filtrado de image_collections para extraer la imagen de interés
test_img = ee.Image(img_collection
                 .filterDate('2024-04-08T18:15:00', '2024-04-08T19:00:00')
                           # Tiempos en UTC (México tiene UTC-6hrs)
                 .first()) # Primer imagen de la lista


### **Image pre-processing**

Al ser un satélite geoestacionario, el ángulo azimutal cambia todo el tiempo, por lo que debemos normalizar las bandas usando este ángulo. Tambien se le aplica un *offset* a la nueva imagen escalada para que no haya distorción.
\
\
Si queremos una imagen RGB, es necesario generar una banda *GREEN* porque el sensor ABI sólo registra *RED* y *BLUE*, por lo que debemos simularla con estas bandas y una banda *veggie* (NIR). [[1](https://doi.org/10.1029/2018EA000379)]

In [ ]:
NUM_BANDS = 33
BLUE_BAND_INDEX = (1 - 1) * 2
RED_BAND_INDEX = (2 - 1) * 2
NIR_BAND_INDEX = (3 - 1) * 2
GREEN_BAND_INDEX = NUM_BANDS - 1

def applyScaleAndOffset(img):

  bands = [0] * NUM_BANDS

  names = img.select('CMI_C..').bandNames()
  for i in range (1, 17):
    num = 100+i
    bandName = 'CMI_C' + str(num)[1:3]
    offset = ee.Number(img.get(bandName + '_offset'))
    scale =  ee.Number(img.get(bandName + '_scale'))
    bands[(i-1) * 2] = img.select(bandName).multiply(scale).add(offset)

    dqfName = 'DQF_C' + str(num)[1:3]
    bands[(i-1) * 2 + 1] = img.select(dqfName)


  green = img.expression(
        ' 0.45 * red + 0.10 * nir + 0.45 * blue ', {
        'nir' : bands[NIR_BAND_INDEX],
        'red' : bands[RED_BAND_INDEX],
        'blue': bands[BLUE_BAND_INDEX]
        }
        ).rename('GREEN')
  bands[GREEN_BAND_INDEX] = green


  return ee.Image(ee.Image(bands).copyProperties(img, img.propertyNames()))

image = applyScaleAndOffset(test_img)

---
---
## **Eclipse path tracking**
---
---

### **Busqueda de mínimo local**

Para saber la trayectoria del eclipse, podemos buscar un mínimo local de la reflectancia de alguna banda espectral (e.j. azul). Para esto, aplicamos un *reducer* sobre una región utilizando **ee.Image.ReduceRegion**, lo cual se observa como:

In [ ]:
# Region Of Interest
ROI = ee.Geometry.BBox(-107.70996093750001, 21.779905342529645,
                        -90.17578125000001, 36.66841891894786)

# Defino las bandas correspondientes a RGB para tener una imagen "true color"
RED  = 'CMI_C02'
GREEN = 'GREEN'
BLUE = 'CMI_C01'

# Función generadora de máscara de la penumbra y su centro
def pxmsk_center(img):

  # Máscara con píxeles con reflectancia nula (oscuridad total)
  cld_min_mask = img.expression(
      'B == 0 ? 1'
      ': 0',
      {
          'B' : img.select(BLUE)
      }

  ).selfMask()

  # Aplicamos un reducer sobre un vecindario alrededor de cada pixel de la máscara
  px_sum = cld_min_mask.reduceNeighborhood(
    reducer = ee.Reducer.count(), # Cuenta el # de píxeles
    kernel = ee.Kernel.circle(110000, units = 'meters') # Radio del círculo alrededor del pixel
  )

  # Buscamos el máximo (idealmente el centro de la nube)
  max = px_sum.reduceRegion(
      reducer = ee.Reducer.max(),
      geometry = ROI,
      scale = 2000,
      crs = 'EPSG:3857'
  ).toArray().get([0])

  # Conseguimos el 'centro' de la nube con una condición.
  center = px_sum.expression(
      "B > max-3000 ? 1 "
      ": 0", {
          'B' : px_sum.select('constant_count'),
          'max' : ee.Image(max)
      }
  ).selfMask()

  return center, cld_min_mask

# La aplicamos a la imagen
cld_center, cld_mask = pxmsk_center(image)

### **Generando el mapa**

Antes de generar el mapa, definimos los parámetros de visualización con nuestra nueva banda verde, para generar una imagen *true-color*.

In [ ]:
rgb_params = {
    'bands': [RED, GREEN, BLUE],
    'min': 0,
    'max': 0.5,
    'gamma': 1.3
}

AOI_params = {
    'color': '00000000',
    'width': 2,
    'lineType': 'solid',
    'fillColor': '00000000',
    'fillColorOpacity': '0'}

Generamos el mapa con estas máscaras

In [ ]:
map3 = geemap.Map()
map3.set_center(-95.8, 32.6, 6)

# RGB
map3.addLayer(image, rgb_params, 'Imagen RGB')

# Paletas de colores para las máscaras
palette = ['red']
palette1 = ['yellow']

# Máscaras
map3.addLayer(cld_mask, {'palette': palette}, ' Zona de penumbra')
map3.addLayer(cld_center, {'palette': palette1}, 'Centro')
map3.addLayer(ROI, AOI_params, 'AOI')
map3

Map(center=[32.6, -95.8], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI…

---
---
## **Animación**
---
---

En esta sección aplicaremos el algoritmo desarrollado sobre una colección de imágenes, con el fin de conseguir una animación.

### **Colección de imágenes**

Filtramos X hrs de imágenes, tomando en cuenta que son

In [ ]:
img_col = img_collection.filterDate('2024-04-08T18:05:00', '2024-04-08T19:00:00')

# Aplicamos la función applyScaleAndOffset
img_col = img_col.map(applyScaleAndOffset)

### **Mapeamos la colección**

Para aplicar una función sobre todas las imágenes de una colección, necesitamos una función de la forma:
```
def function(image):
  new_img = image.algoritmo()
  return new_img

new_col = col.map(function)
```
Usamos la función *ee.ImageCollection.map(function)*.

Por lo que, para generar la animación, necesitamos una función que genere una imagen RGB compuesta de múltiples capas (como las de nuestro mapa).



In [ ]:
# Composite RGB image with multiple layers (non commutative)
def RGBMSKCNTR(img):
  # Imagen RGB
  rgbimg = img.visualize(
      bands = [RED, GREEN, BLUE],
      min = 0,
      max = 0.4,
      gamma = 1.3
      )

  # Conseguimos la máscara de la penumbra y su centro
  center, mask = pxmsk_center(img)

  # Máscara penumbra
  mask = mask.visualize(
      palette = ['red']
  )

  # Máscara centro
  center = center.visualize(
      palette = ['yellow']
  )
  return rgbimg.blend(mask).blend(center)


# Mapeamos nuestra colección con esta nueva función
ani_col = img_col.map(RGBMSKCNTR)

### **Generando el GIF**

In [ ]:
# Le damos parámetros al video
videoArgs = {
  'dimensions': 720,
  'region': ROI,
  'framesPerSecond': 3,
  'crs': 'EPSG:3857',
  }

Mandamos imprimir una URL que nos lleva al video/GIF generado

In [ ]:
def timelist(img):
  #return ee.Date(img).format('H:m', 'America/Mexico_City')
  return ee.Feature(None, {'time': img.date().format('H:m', 'America/Mexico_City')})


times = img_col.map(timelist).aggregate_array('time').getInfo()


In [ ]:
saved_gif = 'eclipse.gif'
geemap.download_ee_video(ani_col, videoArgs, saved_gif)

Generating URL...
Please wait ...
The GIF image has been saved to: /content/eclipse.gif


In [ ]:
text = times
out_gif = 'eclipse_times.gif'
geemap.add_text_to_gif(
    saved_gif,
    out_gif,
    xy=('3%', '5%'),
    text_sequence=text,
    font_size=30,
    font_color='#ffffff',
)